<a href="https://colab.research.google.com/github/vadmbertr/Deep-generative-learning-for-next-generation-drugs/blob/main/transformer_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Charger un modèle de langage pré-entraîné de type transformer (avec son tokéniseur) pour le français à partir du dépôt de HuggingFace :

In [ ]:
import os
import torch
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader, Dataset
from pprint import pprint
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset


HF_MODEL_CACHE_DIR = "c:/Users/ait/.cache/huggingface/transformers"
os.environ['TRANSFORMERS_CACHE'] = HF_MODEL_CACHE_DIR


# French models: ClassCat/roberta-base-french, Yanzhu/bertweetfr-base, Geotrend/distilbert-base-en-fr-es-pt-it-cased, 
pretrained_model_name = "camembert-base" 
# pretrained_model_name = "ClassCat/roberta-base-french"

lmtokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
lm = AutoModel.from_pretrained(pretrained_model_name, output_attentions=False)



Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokéniser le premier texte avec le tokéniseur du modèle de langage :

In [ ]:
texts = [
    "Film gentil et très distrayant. ",
    "J'aime le camembert.",
    "il lit un livre.",
    "il lit le journal.",
    "Des draps et un lit pour dormir.",
    "Il s'allongea sur le lit.",
]

for text in texts:
    tokens = lmtokenizer.tokenize(text, add_special_tokens=True)
    print(tokens)

['<s>', '▁Film', '▁gentil', '▁et', '▁très', '▁dis', 'tr', 'ayant', '.', '</s>']
['<s>', '▁J', "'", 'aime', '▁le', '▁ca', 'member', 't', '.', '</s>']
['<s>', '▁il', '▁lit', '▁un', '▁livre', '.', '</s>']
['<s>', '▁il', '▁lit', '▁le', '▁journal', '.', '</s>']
['<s>', '▁Des', '▁draps', '▁et', '▁un', '▁lit', '▁pour', '▁dormir', '.', '</s>']
['<s>', '▁Il', '▁s', "'", 'allonge', 'a', '▁sur', '▁le', '▁lit', '.', '</s>']


Tokéniser et encoder les textes pour servir d'entrée au modèle de langage pré-entrainé : 

In [ ]:
encoded_texts = lmtokenizer(texts, 
    padding=True,
    add_special_tokens=True,
    return_tensors='pt',
    return_offsets_mapping=False,
)

pprint(encoded_texts)


{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    5,  5897,  4845,    14,    95,   701,  4871,  3276,     9,     6,
             1],
        [    5,   121,    11,   660,    16,   730, 25543,   110,     9,     6,
             1],
        [    5,    51,   849,    23,   510,     9,     6,     1,     1,     1,
             1],
        [    5,    51,   849,    16,  2353,     9,     6,     1,     1,     1,
             1],
        [    5,   363, 16507,    14,    23,   849,    24,  4655,     9,     6,
             1],
        [    5,    69,    52,    11, 12435,    55,    32,    16,   849,     9,
             6]])}


Appliquer le modèle sur les textes et afficher le vecteur de représentation du 2ème mot ("gentil") du premier texte. 

In [ ]:
lm.eval()

# obtenir les vecteurs de représentation contextuels des textes
output = lm.forward(input_ids=encoded_texts['input_ids'], attention_mask=encoded_texts['attention_mask'])[0]

# afficher le vecteur de représentation du 2ème mot du premier texte
output[0][2]



tensor([ 2.7026e-02, -2.2273e-01,  3.4151e-02, -1.2499e-01, -2.5172e-02,
        -5.6220e-02, -1.1643e-05,  1.9083e-02,  5.9024e-02,  5.3655e-02,
         1.2652e-03, -1.4679e-01,  6.7243e-02,  8.0300e-02,  8.1018e-02,
        -2.0575e-03, -2.9184e-02, -7.4812e-02,  2.9311e-03, -5.7235e-02,
        -1.7323e-01,  7.9357e-02,  9.2863e-02, -4.5485e-02, -3.1706e-01,
        -2.3649e-02, -2.6752e-01, -2.8997e-02,  5.0029e-02,  4.0398e-02,
         9.2951e-02,  1.8293e-01,  1.6644e-01, -3.3054e-02,  4.2189e-02,
         5.8270e-02, -3.6130e-02,  1.9843e-01, -2.1253e-02,  3.2097e-01,
        -2.0587e-01,  6.0121e-02,  4.7284e-02, -8.5656e-02,  1.2964e-02,
        -1.1330e-03,  1.5728e-02, -1.6476e-01,  3.8387e-02,  4.3340e-02,
         3.1881e-02, -3.2446e-02, -9.2199e-02,  5.0469e-02, -1.2656e-01,
        -7.8073e-02,  4.5900e-02, -1.8651e-01, -9.5165e-03, -8.5449e-02,
         1.0050e-01, -8.5659e-02,  3.7343e-02, -1.7142e-02, -1.3409e-01,
         1.0619e-02, -2.1054e-01, -3.0255e-02, -3.6

Quelques exemples de scores de similarité entre des mots dans les textes :

In [ ]:

# similarity between word "lit" in text 3 and word "lit" in text 4
print(F.cosine_similarity(output[2][2], output[3][2], dim=0))
# similarity between word "lit" in text 3 and word "lit" in text 5
print(F.cosine_similarity(output[2][2], output[4][5], dim=0))
# etc.
print(F.cosine_similarity(output[4][5], output[5][8], dim=0))
print(F.cosine_similarity(output[2][2], output[5][8], dim=0))


tensor(0.9569, grad_fn=<SumBackward1>)
tensor(0.4915, grad_fn=<SumBackward1>)
tensor(0.7944, grad_fn=<SumBackward1>)
tensor(0.5691, grad_fn=<SumBackward1>)


Exemple de scores de similarité entre les textes entiers, en prenant comme représentant le vecteur du token de début `<s>`:

In [ ]:
# similarity entre phrases
print(F.cosine_similarity(output[2][0], output[3][0], dim=0))
print(F.cosine_similarity(output[2][0], output[4][0], dim=0))
print(F.cosine_similarity(output[4][0], output[5][0], dim=0))

tensor(0.9839, grad_fn=<SumBackward1>)
tensor(0.9541, grad_fn=<SumBackward1>)
tensor(0.9527, grad_fn=<SumBackward1>)


Exemple de scores de similarité entre les textes entiers, en prenant comme représentant la moyenne des vecteurs de tous les tokens d'un texte :

In [ ]:
avgvects = output.mean(dim=1)
print(texts)
# similarity entre phrases
print(F.cosine_similarity(avgvects[2], avgvects[3], dim=0))
print(F.cosine_similarity(avgvects[2], avgvects[4], dim=0))
print(F.cosine_similarity(avgvects[4], avgvects[5], dim=0))

['Film gentil et très distrayant. ', "J'aime le camembert.", 'il lit un livre.', 'il lit le journal.', 'Des draps et un lit pour dormir.', "Il s'allongea sur le lit."]
tensor(0.9443, grad_fn=<SumBackward1>)
tensor(0.7651, grad_fn=<SumBackward1>)
tensor(0.7442, grad_fn=<SumBackward1>)
